In [ ]:
#Loading the data 

In [1]:
import pandas as pd 
import numpy as np

In [6]:
GND = pd.read_csv('C:\\Users\\ritik\\OneDrive\\Documents\\GitHub\\Multi-Model-Bias-Detection-and-Debiasing-the-News\\Good_News_Dataset\\combined_dataset.csv',delimiter=',')

In [8]:
GND.size

2427576

In [9]:
GND.shape[0]

404596

In [10]:
GND.isna().sum()

image_path         0
caption        34813
article            0
headline           2
article_id         0
image_index        0
dtype: int64

In [12]:
gnd_nona = GND.dropna()

In [13]:
gnd_nona.isna().sum()

image_path     0
caption        0
article        0
headline       0
article_id     0
image_index    0
dtype: int64

Lower Casing the whole dataset 

In [14]:
gnd_nona = gnd_nona.applymap(lambda x: x.lower() if isinstance(x, str) else x)


C:\Users\ritik\AppData\Local\Temp\ipykernel_43472\1671697315.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gnd_nona = gnd_nona.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [ ]:
babe['text'] = babe['text'].replace(to_replace=r'[^\w\s]', value='', regex=True)
